# IBM Data Science Certificate Capstone Project

## Boston, Massachusetts Subway Station Analysis

## Import Necessary Libraries for Data Science

In [171]:
import pandas as pd
import numpy as np
import json
import requests
import random
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

## Import MBTA Stop Data to IBM Cloud Storage

In [172]:
# The code was removed by Watson Studio for sharing.

## Clean and Subset Data

In [173]:
drop_columns = ['stop_code', 'stop_url', 'parent_station', 'wheelchair_boarding', 'level_id', 'platform_code', 'vehicle_type', 'location_type', 'on_street', 'at_street', 'stop_address']
mbta_stops = df_data_1.drop(drop_columns, axis=1)
mbta_stops.head()

,stop_id,stop_name,stop_desc,platform_name,stop_lat,stop_lon,zone_id,municipality
0,1,Washington St opp Ruggles St,NaN,NaN,42.330957,-71.082754,ExpressBus-Downtown,Boston
1,10,Theo Glynn Way @ Newmarket Sq,NaN,NaN,42.330555,-71.068787,LocalBus,Boston
2,10000,Tremont St opp Temple Pl,NaN,NaN,42.355692,-71.062911,LocalBus,Boston
3,10003,Albany St opp Randall St,NaN,NaN,42.331591,-71.076237,LocalBus,Boston
4,10005,Albany St opp E Concord St,NaN,NaN,42.335017,-71.071280,LocalBus,Boston


## Use Boolean Filtering to Subset Only Rapid Transit (i.e. Subway Stops)

In [174]:
subway_stops = (mbta_stops['zone_id'] == 'RapidTransit')
subway_stops.head()

0    False
1    False
2    False
3    False
4    False
Name: zone_id, dtype: bool

In [175]:
rapid_transit_stops = mbta_stops[subway_stops].reset_index(drop=True)
rapid_transit_stops.drop_duplicates(subset=['stop_name'], inplace=True)
rapid_transit_stops = rapid_transit_stops.reset_index(drop=True)
rapid_transit_stops.drop(['zone_id', 'platform_name'], axis=1, inplace=True)
rapid_transit_stops.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,municipality
0,70061,Alewife,Alewife - Red Line,42.395428,-71.142483,Cambridge
1,70126,Allston Street,Allston Street - Green Line - Park Street & North,42.348649,-71.137881,Boston
2,70083,Andrew,Andrew - Red Line - Ashmont/Braintree,42.330154,-71.057655,Boston
3,70047,Airport,Airport - Blue Line - Bowdoin,42.374262,-71.030395,Boston
4,70043,Aquarium,Aquarium - Blue Line - Bowdoin,42.359784,-71.051652,Boston


## Install Geocoder and Folium Libraries for Mapping

In [176]:
! pip install geocoder
import geocoder

In [177]:
! pip install folium
import folium

## Pull Boston's GPS Coordinates to Instantiate a Map

In [178]:
city = 'Boston, Massachusetts'
geo = Nominatim(user_agent='mbta_stations')
location = geo.geocode(city)
boston_latitude = location.latitude
boston_longitude = location.longitude

In [179]:
boston_map = folium.Map(location=[boston_latitude, boston_longitude], tiles='CartoDB positron',zoom_start=12)

## Add Color Coded Markers for Each MBTA Subway Station to the Map

In [180]:
for index, row in rapid_transit_stops.iterrows():
    if 'Red' in row['stop_desc'] or 'Mattapan Trolley' in row['stop_desc']:
        folium.CircleMarker([row[3], row[4]], radius=6, weight=1.5, color='red').add_to(boston_map)

for index, row in rapid_transit_stops.iterrows():
    if 'Blue' in row['stop_desc']:
        folium.CircleMarker([row[3], row[4]], radius=6, weight=1.5, color='blue').add_to(boston_map)

for index, row in rapid_transit_stops.iterrows():
    if 'Orange' in row['stop_desc']:
        folium.CircleMarker([row[3], row[4]], radius=6, weight=1.5, color='orange').add_to(boston_map)
        
for index, row in rapid_transit_stops.iterrows():
    if 'stop_name' == 'Green St':
        folium.CircleMarker([row[3], row[4]], radius=6, weight=1.5, color='orange').add_to(boston_map)
    elif 'Green' in row['stop_desc']:
        folium.CircleMarker([row[3], row[4]], radius=6, weight=1.5, color='green').add_to(boston_map)

## View Map of the Region
### Color of the Circle Indicates the Rapid Transit Line (Red, Orange, Blue, and Green)

In [181]:
boston_map

## Get Started with FourSquare API

In [182]:
CLIENT_ID = 'O2CP0BR0AURTH4LGAF1D3LOR2UEFDT4LLPZHLWJ0BETN4SDO' 
CLIENT_SECRET = 'XMHVQ3QEXBY43UDYGXTDV54IOSBS3R43KYAYWILH5ZISUEI3' 
VERSION = '20200618'

#500 meters is approximately one third of a mile which is a comfortable distance to walk in a short period of time (ex. for lunch during work)
radius = 500
limit = 40
query = rapid_transit_stops.loc[0, 'stop_name']
stop_latitude = rapid_transit_stops.loc[0, 'stop_lat']
stop_longitude = rapid_transit_stops.loc[0, 'stop_lon']

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, stop_latitude, stop_longitude, VERSION, query, radius, limit)

In [183]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Test Pull Venue Information for MBTA Stations

In [184]:
station_results = requests.get(url).json()
results_limit_40 = station_results['response']['groups'][0]['items']
nearby_venues = json_normalize(results_limit_40)
column_subset = ['stop_name', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues['stop_name'] = rapid_transit_stops['stop_name']
nearby_venues = nearby_venues.loc[:,column_subset]

In [185]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
ordered_columns = ['name', 'categories', 'lat', 'lng']
nearby_venues = nearby_venues[ordered_columns]
nearby_venues = nearby_venues.rename(columns={'categories':'category', 'lat': 'latitude', 'lng':'longitude'})
remove_metros = (nearby_venues['category'] == 'Metro Station') | (nearby_venues['category'] == 'Bus Station')
nearby_venues = nearby_venues[~remove_metros].reset_index(drop=True)

## Apply Function to Get Venues for All MBTA Stations

In [186]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name, lat, lng, i['venue']['name'], i['venue']['location']['lat'], i['venue']['location']['lng'],  i['venue']['categories'][0]['name']) for i in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['stop_name', 'stop_lat', 'stop_lon', 'venue_name', 'venue_lat', 'venue_lon', 'venue_category']
    return(nearby_venues)

In [187]:
MBTA_venues = getNearbyVenues(rapid_transit_stops['stop_name'], rapid_transit_stops['stop_lat'], rapid_transit_stops['stop_lon'])

In [188]:
drop_rows = ['Metro Station', 'Intersection', 'Border Crossing', 'Airport Terminal', 'Bus Station', 'Airport', 'Boat or Ferry', 'Construction & Landscaping', 'College Cafeteria', 'Souvenir Shop', 'Miscellaneous Shop', 'Tour Provider', 'Neighborhood', 'Bus Line', 'Storage Facility', 'Lawyer', 'College Quad', 'College Arts Building', 'College Rec Center', 'Light Rail Station', 'Platform', 'Gas Station', 'Bus Stop', 'Event Space', 'College Gym', 'College Theater', 'Alternative Healer', 'Train Station', 'Transportation Service', 'Hotel Pool']
MBTA_venues = MBTA_venues[~MBTA_venues['venue_category'].isin(drop_rows)]
MBTA_venues.head()

,stop_name,stop_lat,stop_lon,venue_name,venue_lat,venue_lon,venue_category
0,Alewife,42.395428,-71.142483,Minuteman Commuter Bikeway,42.397172,-71.143305,Trail
1,Alewife,42.395428,-71.142483,Revival Cafe,42.395256,-71.145900,Café
2,Alewife,42.395428,-71.142483,Summer Shack,42.393682,-71.140770,Seafood Restaurant
3,Alewife,42.395428,-71.142483,Alewife Reservation,42.397205,-71.140445,Park
4,Alewife,42.395428,-71.142483,Alewife Brook Greenway,42.397647,-71.143624,Trail


## Categorize and Count Venue Types

In [189]:
groups = ['food/drink', 'retail', 'services', 'outdoors/exercise', 'nightlife']

food = ['Seafood Restaurant', 'Food Truck', 'Donut Shop', 'Pizza Place', 'Coffee Shop', 'Polish Restaurant', 'Restaurant', 'Chinese Restaurant', 'Italian Restaurant', \
        'Brazilian Restaurant', 'Latin American Restaurant', 'Ice Cream Shop', 'Bakery', 'Colombian Restaurant', 'American Restaurant', 'Greek Restaurant', 'Mexican Restaurant', 'Deli / Bodega', \
        'Belgian Restaurant', 'Sandwich Place', 'French Restaurant', 'Sushi Restaurant', 'Tapas Restaurant', 'Steakhouse', 'Korean Restaurant', 'Café', 'Udon Restaurant', 'Dessert Shop', \
        'Israeli Restaurant', 'Japanese Restaurant', 'Food', 'Market', 'Cupcake Shop', 'Pastry Shop', 'Southern / Soul Food Restaurant', 'BBQ Joint', 'Tea Room', 'Candy Store', \
        'Bubble Tea Shop', 'Thai Restaurant', 'Fried Chicken Joint', 'Taco Place', 'Food Court', 'New American Restaurant', 'Breakfast Spot', 'Food & Drink Shop', 'Juice Bar', \
       'Salad Place', 'Burger Joint', 'Indian Restaurant', 'Burrito Place', 'Middle Eastern Restaurant', 'Asian Restaurant', 'Noodle House', 'Australian Restaurant', 'Diner', 'Hot Dog Joint', 'Falafel Restaurant', \
       'Caribbean Restaurant', 'Vegetarian / Vegan Restaurant', 'Fast Food Restaurant', 'Bagel Shop', 'Mediterranean Restaurant', 'Hotpot Restaurant', 'Ethiopian Restaurant', 'Tibetan Restaurant', \
        'Shanghai Restaurant', 'Chocolate Shop', 'Bistro', 'Creperie', 'Soba Restaurant', 'Poke Place', 'Eastern European Restaurant', 'Frozen Yogurt Shop', 'Dumpling Restaurant', 'Arepa Restaurant', \
       'Ramen Restaurant', 'Comfort Food Restaurant', 'Taiwanese Restaurant', 'Szechuan Restaurant', 'South American Restaurant', 'Venezuelan Restaurant', 'Cuban Restaurant',  'Cajun / Creole Restaurant', \
        'African Restaurant', 'Wings Joint', 'Peruvian Restaurant', 'Vietnamese Restaurant']

retail = ['Flower Shop', 'Cosmetics Shop', 'Department Store', 'Electronics Store', 'Furniture / Home Store', 'Shopping Mall', 'Arts & Crafts Store', 'Jewelry Store', 'Wine Shop', \
          'Clothing Store', 'Big Box Store', 'Mobile Phone Shop', 'Convenience Store', 'Mattress Store', 'Gourmet Shop', 'Lingerie Store', 'Sporting Goods Shop', 'Boutique', "Women's Store", \
         'Record Shop', 'Thrift / Vintage Store', 'Shoe Store', 'Bookstore', 'Music Store', 'Discount Store', 'Hardware Store', 'Toy / Game Store', 'Pet Store', 'Garden Center', "Men's Store", 'Gift Shop', \
          'Kids Store', 'Board Shop', 'Hobby Shop', 'Comic Shop', 'Beer Store', 'Warehouse Store']

service = ['Rental Car Location', 'Paper / Office Supplies Store', 'Hotel', 'Hostel', 'Spa', 'Bank', 'Video Store', 'Grocery Store', 'Liquor Store', 'Pharmacy', 'Supermarket', 'Health & Beauty Service', \
          'Nail Salon', 'Optical Shop', 'Dry Cleaner', 'Automotive Shop', 'Athletics & Sports', 'Shipping Store', 'Post Office', 'Salon / Barbershop', 'ATM', 'Other Repair Shop', 'Pet Service', "Doctor's Office",\
          'Tattoo Parlor', 'Smoke Shop', 'Business Service', 'Bed & Breakfast', 'Tanning Salon', 'Church']

outside = ['Pool', 'Park', 'Trail', 'Plaza', 'Dog Run', 'Gym', 'Pilates Studio', 'Playground', 'Historic Site', 'Scenic Lookout', 'Bike Rental / Bike Share', 'Baseball Field', 'Gym / Fitness Center', \
           'Aquarium', 'Garden', 'Boxing Gym', 'Yoga Studio', 'Farmers Market', 'Lake', 'Pedestrian Plaza', 'River', 'Martial Arts Dojo', 'Cycle Studio', 'Track', 'Skate Park', 'Harbor / Marina', 'Tennis Court', \
          'Sculpture Garden', 'Monumnet / Landmark', 'Track Stadium', 'Beach', 'Skating Rink', 'Hockey Arena', 'Outdoor Sculpture', 'Soccer Field', 'Squash Court', 'Golf Course']

nightlife = ['Bar', 'Dive Bar', 'Pub', 'Beer Garden', 'Comedy Club', 'Lounge', 'Gastropub', 'Baseball Stadium', 'Opera House', 'Movie Theater', 'Performing Arts Venue', 'Brewery', 'Bowling Alley', 'Theater', \
            'Rock Club', 'Cocktail Bar', 'Sports Bar', 'Roof Deck', 'Nightclub', 'Circus', 'Music Venue', 'Museum', 'Art Gallery', 'History Museum', 'Marijuana Dispensary', 'Speakeasy', 'Wine Bar', 'Hotel Bar',\
            'Indie Movie Theater', 'Whisky Bar', 'Jazz Club', 'Social Club', 'Concert Hall', 'Art Museum', 'Karaoke Bar', 'Sake Bar', 'Community Center', 'Arcade', 'Recording Studio', 'Arts & Entertainment', \
             'General Entertainment']

MBTA_venues['group'] = np.nan

for row, index in MBTA_venues.iterrows():
    if MBTA_venues.loc[row, 'venue_category'] in food:
        MBTA_venues.loc[row, 'group'] = groups[0]
    elif MBTA_venues.loc[row, 'venue_category'] in retail:
        MBTA_venues.loc[row, 'group'] = groups[1]
    elif MBTA_venues.loc[row, 'venue_category'] in service:
        MBTA_venues.loc[row, 'group'] = groups[2]
    elif MBTA_venues.loc[row, 'venue_category'] in outside:
        MBTA_venues.loc[row, 'group'] = groups[3]
    elif MBTA_venues.loc[row, 'venue_category'] in nightlife:
        MBTA_venues.loc[row, 'group'] = groups[4]

## Get the Size of the Venues DataFrame

In [190]:
print(MBTA_venues.shape)

(2922, 8)


In [191]:
MBTA_venues = MBTA_venues.reset_index(drop=True)
MBTA_venues.head()

,stop_name,stop_lat,stop_lon,venue_name,venue_lat,venue_lon,venue_category,group
0,Alewife,42.395428,-71.142483,Minuteman Commuter Bikeway,42.397172,-71.143305,Trail,outdoors/exercise
1,Alewife,42.395428,-71.142483,Revival Cafe,42.395256,-71.145900,Café,food/drink
2,Alewife,42.395428,-71.142483,Summer Shack,42.393682,-71.140770,Seafood Restaurant,food/drink
3,Alewife,42.395428,-71.142483,Alewife Reservation,42.397205,-71.140445,Park,outdoors/exercise
4,Alewife,42.395428,-71.142483,Alewife Brook Greenway,42.397647,-71.143624,Trail,outdoors/exercise


## Pivot the DataFrame to Easily View a Count of Venues by Station

In [192]:
table = pd.pivot_table(MBTA_venues, values='stop_lat', index=['stop_name'],
                    columns=['group'], aggfunc='count')
table.head()

group,food/drink,nightlife,outdoors/exercise,retail,services
stop_name,,,,,
Airport,10.0,1.0,4.0,1.0,1.0
Alewife,7.0,NaN,9.0,NaN,1.0
Allston Street,5.0,3.0,2.0,1.0,2.0
Andrew,10.0,2.0,1.0,8.0,3.0
Aquarium,19.0,4.0,11.0,NaN,4.0


## Replace NaN Values with 0, Add a Total Column, and Sort the Table on Total (Highest to Lowest)

In [206]:
venues_table = table.fillna(0)
venues_table['total'] = venues_table.sum(axis=1)
venues_table = venues_table.sort_values(['total'], ascending=False)
venues_table.head()

group,food/drink,nightlife,outdoors/exercise,retail,services,total
stop_name,,,,,,
Coolidge Corner,22.0,2.0,5.0,7.0,4.0,40.0
Harvard Avenue,29.0,4.0,2.0,3.0,2.0,40.0
Harvard,20.0,6.0,1.0,10.0,3.0,40.0
South Station,28.0,5.0,4.0,1.0,2.0,40.0
Griggs Street,24.0,6.0,2.0,4.0,4.0,40.0


## Add Markers to the Map for Visual Aid

In [194]:
for index, row in MBTA_venues.iterrows():
    folium.CircleMarker([row[4], row[5]], radius=3, weight=1, color='black',tooltip=row[3]).add_to(boston_map)

In [195]:
boston_map